In [ ]:
# Installations
!pip install pyLDAvis # Install pyLDAvis
!pip install -U gensim # Colab's version of gensim is deprecated 

     |████████████████████████████████| 24.2MB 41.9MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
# Import required Module
import pandas as pd
from datetime import datetime
import re
from collections import Counter
import numpy as np
from gensim import corpora
from gensim.models import ldamodel
import pyLDAvis.gensim as gensimvis
import pyLDAvis
import pickle

In [ ]:
# Getting required methods
from gensim.parsing.preprocessing import STOPWORDS

In [ ]:
# Set random seed 
np.random.seed(2)

In [ ]:
# Function to import csv file's reviews and convert it to raw data 
def load_csv_data(filename):
    data = pd.read_csv(filename, error_bad_lines=False, encoding='latin1', names = range(18))
    data = data.rename(columns = {15 : 'Reviews'}) # Change this line if your csv file has column headers
    rawtext = data['Reviews'].tolist()
    return rawtext

In [ ]:
# Function to load in Frequent Word Text File
"""" 
def loadFrequentWords(freqWordFile):
    file = open(freqWordFile,'r')
    frequentWords = []
    for word in file:
        frequentWords.append(word.replace("\n",""))
    return frequentWords
""""

In [ ]:
# Function to preprocess text 
def pre_process(text, frequent_words):
  pre_processed = []
  for line in text:
    words = str(line).split(' ')
    pre_processed.append([word for word in words if word.strip().lower().replace("'",'').replace('.', '') not in frequent_words and word !="" and len(word) > 2])
  
  return pre_processed

In [ ]:
# Modeling Function 
def topicmodeling(preprocessed_text_list,num_topics=10,iterations =50):
    dictionary = corpora.Dictionary(preprocessed_text_list)
    corpus = [dictionary.doc2bow(text) for text in preprocessed_text_list]
    lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary,num_topics=num_topics, update_every=1, iterations =iterations,random_state=2)  
    return (lda, dictionary, corpus)

In [ ]:
# Ignore the last two lines of code if you have a custom frequently used words text file
file_name = '/content/drive/MyDrive/OBGYN.csv' 
# freq_word_file = 'file path' 
other_words = ['dr', 'doctor', 'im', 'didnt', 'ive' 'dont', 'wont']
freq_words = STOPWORDS.union(set(other_words))

In [ ]:
# Create list of pre processed text using the above functions
pre_processed_text = pre_process(load_csv_data(file_name), freq_words)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (7,9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
# Running the LDA model
num_topics=10
iterations =3000
lda, dictionary, corpus = topicmodeling(pre_processed_text,num_topics=num_topics,iterations=iterations)

In [ ]:
# Visualizing
pyLDAvis.enable_notebook(local=True)
vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary, R = 30)

In [ ]:
# Saving Visualization as an html file
# Note: you can use the .display method to show the plot in the notebook but the bars will not be labeled
pyLDAvis.display(vis_data)
pyLDAvis.save_html(vis_data,"vis_lda.html")